In [1]:
%pip install geopy
%pip install fuzzywuzzy
%pip install folium
%pip install ipywidgets
%pip install streamlit
%pip install streamlit-folium

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip in

In [2]:
# Import Libraries
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import folium
from folium import Popup
from folium import IFrame
from folium import FeatureGroup
import random
import ipywidgets as widgets
from ipywidgets import interactive
from ipywidgets import interact
from IPython.display import display


/Users/jonathan/Library/Python/3.9/lib/python/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/Users/jonathan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Read 2024 Stocking Recepits csv
df = pd.read_csv('2024 Stocking Recepits.csv')
df.head()

,Unnamed: 0,COUNTY,DATE,WATER,TOWN,SPECIES,QTY,SIZE (inch)
0,0,Androscoggin,2024-01-23,RANGE P (MIDDLE),Poland,BROOK TROUT,100,14
1,1,Androscoggin,2024-02-15,RANGE P (LOWER),Poland,BROOK TROUT,300,14
2,2,Androscoggin,2024-04-02,WORTHLEY P,Poland,BROOK TROUT,200,10
3,3,Androscoggin,2024-04-11,BARTLETT P,Livermore,BROOK TROUT,440,10
4,4,Androscoggin,2024-04-11,BRETTUNS P,Livermore,BROOK TROUT,300,10


In [4]:
df = df.drop(columns=['Unnamed: 0'])
df.head(15)

,COUNTY,DATE,WATER,TOWN,SPECIES,QTY,SIZE (inch)
0,Androscoggin,2024-01-23,RANGE P (MIDDLE),Poland,BROOK TROUT,100,14
1,Androscoggin,2024-02-15,RANGE P (LOWER),Poland,BROOK TROUT,300,14
2,Androscoggin,2024-04-02,WORTHLEY P,Poland,BROOK TROUT,200,10
3,Androscoggin,2024-04-11,BARTLETT P,Livermore,BROOK TROUT,440,10
4,Androscoggin,2024-04-11,BRETTUNS P,Livermore,BROOK TROUT,300,10
5,Androscoggin,2024-04-11,ROUND P,Livermore,BROOK TROUT,200,10
6,Androscoggin,2024-04-18,ANDROSCOGGIN R,Lisbon,BROOK TROUT,300,10
7,Androscoggin,2024-04-18,BEAVER PARK P #2,Lisbon,BROOK TROUT,100,10
8,Androscoggin,2024-04-18,BEAVER PARK P #3,Lisbon,BROOK TROUT,100,10
9,Androscoggin,2024-04-18,MEADOW B,Durham,BROOK TROUT,100,10


In [5]:
coordsdf = pd.read_csv('file.csv')

In [6]:
# Extract unique combinations of WATER and TOWN
unique_locations = df[['WATER', 'TOWN']].drop_duplicates()
unique_locations

,WATER,TOWN
0,RANGE P (MIDDLE),Poland
1,RANGE P (LOWER),Poland
2,WORTHLEY P,Poland
3,BARTLETT P,Livermore
4,BRETTUNS P,Livermore
...,...,...
1768,PARKER (BARKER) P,Lyman
1772,YORK P,Eliot
1773,ALEWIFE P,Kennebunk
1784,SQUARE P,Shapleigh


In [7]:
def fuzzy_match_with_coordinates(df, coordsdf, unique_locations):
    coordinates = {}
    discard_coord = []

    # Iterate over the unique combinations of WATER and TOWN
    for _, row in unique_locations.iterrows():
        water_name = row['WATER']
        town_name = row['TOWN']

        # Combine water and town for a fuzzy match
        combined_name = f"{water_name} {town_name}"

        # Find the best match for the combined name
        match = process.extractOne(combined_name, coordsdf['name'], scorer=fuzz.token_sort_ratio)
        
        # Extract the best match, score, and index (3 values returned)
        best_match = match[0]  # The matched string
        score = match[1]       # The similarity score
        index = match[2]       # The index of the matched string (not used in this case)

        # Only proceed if the score is 60 or above
        if score >= 60:
            # Get the corresponding coordinates
            matched_row = coordsdf.iloc[index]
            x_coord = matched_row['X']
            y_coord = matched_row['Y']
            coordinates[(water_name, town_name)] = (x_coord, y_coord)
        else:
            # Store the unmatched rows in discard_coord with the water, town, and score
            discard_coord.append({'Water': water_name, 'Town': town_name, 'Score': score})

    # Create discard_df to show unmatched rows
    discard_df = pd.DataFrame(discard_coord)
    print("Rows with unmatched coordinates:")
    print(discard_df)

    # Now assign the coordinates to all rows in df based on WATER and TOWN combinations
    df['coordinates'] = df.apply(lambda row: coordinates.get((row['WATER'], row['TOWN'])), axis=1)

    # Replace missing coordinates with (44.6939, -69.3815)
    df['coordinates'] = df['coordinates'].apply(lambda x: (44.6939, -69.3815) if x is None else x)

    # Extract X and Y from coordinates (handle missing ones that are replaced)
    df['X_coord'], df['Y_coord'] = zip(*df['coordinates'])

    # Drop the temporary coordinates column
    df = df.drop(columns=['coordinates'])

    return df, discard_df  # Return both the modified df and discard_df

# Get the coordinates for each row in df
df, discard_df = fuzzy_match_with_coordinates(df, coordsdf, unique_locations)

# Display the updated DataFrame
print(df)

# You can access and print discard_df at any time
print(discard_df)  # Prints the discard_df DataFrame


Rows with unmatched coordinates:
                       Water            Town  Score
0                   MEADOW B          Durham     56
1                   NEWELL B          Durham     53
2                    ROYAL R          Auburn     54
3    ANDROSCOGGIN R (LITTLE)          Auburn     57
4    ANDROSCOGGIN R (LITTLE)  Mechanic Falls     53
..                       ...             ...    ...
152                   WEST B           Wells     54
153                 LITTLE R          Buxton     58
154                ELL (L) P           Wells     53
155                 MOUSAM L       Shapleigh     55
156                STEVENS B           Wells     51

[157 rows x 3 columns]
            COUNTY        DATE               WATER       TOWN      SPECIES  \
0     Androscoggin  2024-01-23    RANGE P (MIDDLE)     Poland  BROOK TROUT   
1     Androscoggin  2024-02-15     RANGE P (LOWER)     Poland  BROOK TROUT   
2     Androscoggin  2024-04-02          WORTHLEY P     Poland  BROOK TROUT   
3     A

In [8]:
discard_df['Water'].unique()

array(['MEADOW B', 'NEWELL B', 'ROYAL R', 'ANDROSCOGGIN R (LITTLE)',
       'NEZINSCOT R', 'LITTLE R', 'BROWN P', 'MATTAWAMKEAG R',
       'MATTAWAMKEAG R (EAST BRANCH)', 'CORBIN P',
       'MEDUXNEKEAG R (NORTH BRANCH)', 'CHURCH ON THE HILL P',
       'FISH RIVER', 'SCOPAN STREAM', 'BRANDY B', 'COLLYER B',
       'CHANDLER B', 'PISCATAQUA R', 'SONGO R', 'LITTLE R (NORTH BRANCH)',
       'PLEASANT R', 'PRESUMPSCOT R', 'JORDAN R', 'MUDDY R',
       'NORTHWEST R', 'LONG L', 'MUDDY B', 'CARRABASSETT R', 'SANDY R',
       'WEBB R', 'DEAD R (SOUTH BRANCH)', 'MT BLUE HIGH SCHOOL P',
       'TEMPLE STREAM', 'PATTEN P (LOWER)', 'BELGRADE STREAM',
       'ECHO L (CROTCHED P)', 'SALMON L (ELLIS P)', 'MESSALONSKEE STREAM',
       'SAM ASSOCIATION P (LOWER SMALL)', 'SAM ASSOCIATION P (UPPER BIG)',
       'CHINA L', 'WILSON STREAM', 'MONMOUTH SPORTSMAN CLUB P',
       'WAYNE F&G CLUB P', 'JUG STREAM', 'LONG P', 'GOOSE R',
       'MEGUNTICOOK R', 'MONTSWEAG B', 'WALDOBORO QUARRY P',
       'AUNT HAN

In [9]:
# Check for rows where coordinates are missing (NaN)
missing_coords = df[df['X_coord'].isna() | df['Y_coord'].isna()]
print(f"Rows with missing coordinates: {missing_coords}")


Rows with missing coordinates: Empty DataFrame
Columns: [COUNTY, DATE, WATER, TOWN, SPECIES, QTY, SIZE (inch), X_coord, Y_coord]
Index: []


In [10]:
# Check for invalid coordinates
invalid_coords = df[(df['X_coord'] < -180) | (df['X_coord'] > 180) | 
                     (df['Y_coord'] < -90) | (df['Y_coord'] > 90)]
print(f"Rows with invalid coordinates: {invalid_coords}")


Rows with invalid coordinates: Empty DataFrame
Columns: [COUNTY, DATE, WATER, TOWN, SPECIES, QTY, SIZE (inch), X_coord, Y_coord]
Index: []


In [11]:
df.to_csv('2024_Stocking_Coords.csv')

In [12]:
coordfixdf = pd.read_csv('CoordFixes.csv')
coordfixdf

,WATER,TOWN,Y_coord,X_coord
0,TROUT POND,Mason Twp,44.370567,-70.874295
1,SUNDAY RIVER,Newry,44.490453,-70.838850
2,ELLIS R,Andover,44.613540,-70.720524
3,ELLIS R,Rumford,44.523153,-70.673767
4,ELLIS R (WEST BRANCH),Andover,44.611733,-70.733100
...,...,...,...,...
238,LITTLE R,Buxton,43.670197,-70.536234
239,ELL (L) P,Wells,43.345794,-70.715335
240,MOUSAM L,Shapleigh,43.535682,-70.863212
241,STEVENS B,Wells,43.290746,-70.600527


In [13]:
print(coordfixdf.columns)


Index(['WATER', 'TOWN', 'Y_coord', 'X_coord'], dtype='object')


In [14]:
def update_coordinates(main_df, coordfix_df):
    # Merge the main df with the coordfixdf on 'WATER' and 'TOWN'
    merged_df = pd.merge(main_df, coordfix_df[['WATER', 'TOWN', 'X_coord', 'Y_coord']], on=['WATER', 'TOWN'], how='left', suffixes=('_main', '_corrected'))
    
    # Update the coordinates in the main df with the corrected coordinates
    merged_df['X_coord'] = merged_df['X_coord_corrected'].combine_first(merged_df['X_coord_main'])
    merged_df['Y_coord'] = merged_df['Y_coord_corrected'].combine_first(merged_df['Y_coord_main'])
    
    # Drop the unnecessary columns and return the updated dataframe
    merged_df = merged_df.drop(columns=['X_coord_main', 'Y_coord_main', 'X_coord_corrected', 'Y_coord_corrected'])
    
    return merged_df

# Example usage
df_updated = update_coordinates(df, coordfixdf)


In [15]:
df.head()

,COUNTY,DATE,WATER,TOWN,SPECIES,QTY,SIZE (inch),X_coord,Y_coord
0,Androscoggin,2024-01-23,RANGE P (MIDDLE),Poland,BROOK TROUT,100,14,-70.379485,44.026658
1,Androscoggin,2024-02-15,RANGE P (LOWER),Poland,BROOK TROUT,300,14,-70.369148,44.043731
2,Androscoggin,2024-04-02,WORTHLEY P,Poland,BROOK TROUT,200,10,-70.344777,44.018762
3,Androscoggin,2024-04-11,BARTLETT P,Livermore,BROOK TROUT,440,10,-70.205208,44.401191
4,Androscoggin,2024-04-11,BRETTUNS P,Livermore,BROOK TROUT,300,10,-70.250268,44.392663


In [16]:
# Initialize the map, set to a central point in Maine (or adjust as needed)
map_center = [44.6939, -69.3815]  # Central Maine coordinates (latitude, longitude)
m = folium.Map(location=map_center, zoom_start=7)

# Group data by water body, town, and county
grouped = df_updated.groupby(['WATER', 'TOWN', 'COUNTY'])

# Function to update the map based on selected species, date filters (Spring/Fall), and search
def update_map(selected_species, show_spring, show_fall, search_term, min_qty):
    # Clear the map before adding new markers
    m = folium.Map(location=map_center, zoom_start=7)

    # Filter the grouped data based on the search term
    filtered_groups = []
    for (water_name, town_name, county_name), group in grouped:
        # Check if the search term is in any of the fields (water, town, or county)
        if (search_term.lower() in water_name.lower() or
            search_term.lower() in town_name.lower() or
            search_term.lower() in county_name.lower()):
            filtered_groups.append((water_name, town_name, county_name, group))

    # Add markers to the map for each unique water body, town, and county combination
    for (water_name, town_name, county_name, group) in filtered_groups:
        species_list = []
        qty_list = []
        size_list = []
        date_list = []

        # Loop through each row in the group and collect the data
        for _, row in group.iterrows():
            species_list.append(row['SPECIES'])
            qty_list.append(row['QTY'])
            size_list.append(row['SIZE (inch)'])
            date_list.append(row['DATE'])

        # Filter data by species and date (Spring/Fall)
        filtered_rows = []
        for species, qty, size, date in zip(species_list, qty_list, size_list, date_list):
            month = pd.to_datetime(date).month  # Get the month from the date
            if (show_spring and month <= 6) or (show_fall and month >= 7):
                if species in selected_species and qty >= min_qty:  # Filter by min_qty
                    filtered_rows.append((species, qty, size, date))

        # Only display markers if there is data to show after filtering
        if filtered_rows:
            popup_text = f"""
            <b>Water Body:</b> {water_name}<br>
            <b>Town:</b> {town_name}<br>
            <b>County:</b> {county_name}<br>
            <b>Stocking Data:</b><br>
            <ul>
            """
            for species, qty, size, date in filtered_rows:
                popup_text += f"""
                <li><b>{species}</b> - {qty} fish, Size: {size} inches, Date: {date}</li>
                """
            popup_text += "</ul>"

            # Use the average coordinates of the water body/town combination
            avg_x = group['X_coord'].mean()
            avg_y = group['Y_coord'].mean()

            # Offset coordinates slightly if there are multiple towns associated with the same water body
            offset_x = random.uniform(-0.001, 0.001)  # Random offset for longitude
            offset_y = random.uniform(-0.001, 0.001)  # Random offset for latitude

            # Add a single marker for the water body/town combo with grouped popup data
            folium.Marker(
                location=[avg_y + offset_y, avg_x + offset_x],  # Apply offset to average coordinates
                popup=folium.Popup(popup_text, max_width=300),
                icon=folium.Icon(color='green')  # Popup with the grouped stocking data
            ).add_to(m)

    # Display the updated map
    display(m)

# List of unique species in your dataset
species_list = df_updated['SPECIES'].unique()

# Create checkboxes for each species
checkboxes = widgets.SelectMultiple(
    options=species_list,
    value=species_list.tolist(),  # Start with all species selected
    description='Species:',
    disabled=False
)

# Create checkboxes for Spring and Fall Stocking
spring_checkbox = widgets.Checkbox(
    value=False,
    description='Spring Stocking (Jan-Jun)',
    disabled=False
)

fall_checkbox = widgets.Checkbox(
    value=False,
    description='Fall Stocking (Jul-Dec)',
    disabled=False
)

# Create a search bar to filter by water body, town, or county
search_bar = widgets.Text(
    value='',
    description='Search:',
    placeholder='Type water body, town, or county'
)

# Create a text box for users to enter the minimum quantity of fish
qty_textbox = widgets.IntText(
    value=0,  # Default value
    description='Min Qty:',
    min=0,  # Minimum allowed value
    step=1,  # Step size for the input
    style={'description_width': 'initial'}
)

# Display the checkboxes, text box, and search bar, and link them to the map update function
interact(update_map, 
         selected_species=checkboxes, 
         show_spring=spring_checkbox, 
         show_fall=fall_checkbox, 
         search_term=search_bar, 
         min_qty=qty_textbox)

# Save the map to an HTML file (optional)
m.save("FishMap_2024.html")


interactive(children=(SelectMultiple(description='Species:', index=(0, 1, 2, 3, 4, 5), options=('BROOK TROUT',…

In [17]:
df_updated.to_csv("df_updated.csv", index=False)
